# Hosting Strands Agents with Amazon Bedrock models in Amazon Bedrock AgentCore Runtime

## Overview

In this tutorial we will learn how to host your existing agent, using Amazon Bedrock AgentCore Runtime. We will provide examples using Amazon Bedrock models and non-Bedrock models such as Azure OpenAI and Gemini.


### Tutorial Details


| Information         | Details                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| Tutorial type       | Conversational                                                                   |
| Agent type          | Single                                                                           |
| Agentic Framework   | Strands Agents                                                                   |
| LLM model           | Anthropic Claude Sonnet 3.7                                                        |
| Tutorial components | Hosting agent on AgentCore Runtime. Using Strands Agent and Amazon Bedrock Model |
| Tutorial vertical   | Cross-vertical                                                                   |
| Example complexity  | Easy                                                                             |
| SDK used            | Amazon BedrockAgentCore Python SDK and boto3                                     |

### Tutorial Architecture

In this tutorial we will describe how to deploy an existing agent to AgentCore runtime. 

For demonstration purposes, we will  use a Strands Agent using Amazon Bedrock models

In our example we will use a very simple agent with two tools: `get_weather` and `get_time`. 

<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="50%"/>
</div>

### Tutorial Key Features

* Hosting Agents on Amazon Bedrock AgentCore Runtime
* Using Amazon Bedrock models
* Using Strands Agents


## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* Strands Agents

In [48]:
!pip install -U -r requirements.txt --quiet

## Creating your agents and experimenting locally

Before we deploy our agents to AgentCore Runtime, let's develop and run them locally for experimentation purposes.

For production agentic applications we will need to decouple the agent creation process from the agent invocation one. With AgentCore Runtime, we will decorate the invocation part of our agent with the `@app.entrypoint` decorator and have it as the entry point for our runtime. Let's first look how each agent is developed during the experimentation phase.

This is a sample agent code:

<div style="text-align:left">
    <img src="images/architecture_local.png" width="50%"/>
</div>

In [ ]:
import os
import sys
import boto3
from strands import Agent, tool
from strands.models import BedrockModel
from bedrock_agentcore.runtime import BedrockAgentCoreApp
import tempfile
from datetime import datetime
from rich.console import Console
import time
import glob
import shutil
import ast
import json

app = BedrockAgentCoreApp()

# Set AWS region
REGION = os.environ.get('AWS_REGION', 'us-east-1')

# Create an SSM client
ssm = boto3.client('ssm', region_name=REGION)
response = ssm.get_parameter(
    Name="NOVA_ACT_API_KEY",
    WithDecryption=True
)
nova_act_api_key = response['Parameter']['Value']
print(f"✅ Nova Act API Key retrieved")

# tool to perform web automation and download files using Nova Act
@tool
def nova_act_download(instruction: str, starting_url: str, client_name: str):
    """Download files from websites using Nova Act automation
    
    Args:
        instruction: The task to perform (including login and actions)
        starting_url: The website URL to start automation
        client_name: Client identifier for S3 organization
    
    Returns:
        dict: Status and file information or error details
    """
    # Import inside function to avoid pydantic conflicts
    try:
        from nova_act import NovaAct
    except Exception as import_error:
        return {"status": "error", "reason": f"Failed to import NovaAct: {str(import_error)}"}

    console = Console()
    download_dir = tempfile.gettempdir()
    file_path = None
    download_triggered = False

    try:
        with NovaAct(
            headless=True,
            nova_act_api_key=nova_act_api_key,
            starting_page=starting_url
        ) as nova_act:
            
            prompt = """You are a helpful Web UI automation assistant.
SYSTEM PROMPT:
- Even the task cannot be completed, always return ACTION COMPLETE, never return error !!!!!
- YOU SHOULD NEVER REPEAT THE SAME ACTION MORE THAN ONCE. If your action is unsuccessful, return ACTION COMPLETE !!!!!
- After clicked on download, even if the page looks like it does not change, immediately return ACTION COMPLETE !!!!!
- YOU SHOULD NEVER DISPLAY the password in plan task during execution. 

USER PROMPT: 

""" + instruction
            
            console.print("[cyan]Starting NovaAct automation...[/cyan]")
            result = nova_act.act(prompt)
            console.print(result)
                
            # Check file system for recent downloads
            possible_download_dirs = []
            temp_base = tempfile.gettempdir()
            playwright_dirs = glob.glob(os.path.join(temp_base, "playwright-*"))
            
            for temp_dir in playwright_dirs:
                downloads_subdir = os.path.join(temp_dir, "downloads")
                if os.path.exists(downloads_subdir):
                    possible_download_dirs.append(downloads_subdir)
                    console.print(f"[cyan]Found Playwright downloads dir: {downloads_subdir}[/cyan]")
            
            for temp_dir in playwright_dirs:
                if os.path.exists(temp_dir):
                    possible_download_dirs.append(temp_dir)
            
            possible_download_dirs.append(download_dir)
            
            current_time = time.time()
            recent_files = []
            
            for location in possible_download_dirs:
                if os.path.exists(location):
                    all_files = glob.glob(os.path.join(location, "*"))
                    for f in all_files:
                        if os.path.isfile(f):
                            file_age = current_time - os.path.getmtime(f)
                            if file_age < 45:
                                recent_files.append((f, os.path.getmtime(f)))
                                console.print(f"[cyan]  ✓ Found recent file ({file_age:.1f}s old): {os.path.basename(f)}[/cyan]")
            
            if recent_files:
                recent_files.sort(key=lambda x: x[1], reverse=True)
                most_recent_file = recent_files[0][0]
                console.print(f"[green]✅ Found downloaded file: {most_recent_file}[/green]")
                
                file_path = os.path.join(download_dir, os.path.basename(most_recent_file))
                shutil.copy2(most_recent_file, file_path)
                download_triggered = True
                console.print(f"Copied to: {file_path}")
            else:
                console.print("[red]No recent files found in download directories[/red]")
            
            if not file_path:
                console.print("[yellow]No download detected, trying expect_download...[/yellow]")
                try:
                    with nova_act.page.expect_download(timeout=5000) as download_info:
                        result = nova_act.act("Click the download button once and IMMEDIATELY RETURN ACTION COMPLETE")
                    
                    if download_info.value:
                        console.print("[green]✅ Download event captured[/green]")
                        original_filename = download_info.value.suggested_filename
                        if callable(original_filename):
                            original_filename = original_filename() or "downloaded_file"
                        
                        file_path = os.path.join(download_dir, original_filename)
                        download_info.value.save_as(file_path)
                        console.print(f"Downloaded via event: {file_path}")
                except TimeoutError:
                    console.print("[yellow]No download event detected[/yellow]")
                except Exception as e:
                    console.print(f"[yellow]Download event check failed: {e}[/yellow]")
            
            if file_path and os.path.exists(file_path):
                file_size = os.path.getsize(file_path)
                file_ext = os.path.splitext(file_path)[1]
                
                if file_size == 0:
                    console.print("[red]Downloaded file is empty[/red]")
                    return {"status": "error", "reason": "File is empty"}
                
                if file_ext == '.crdownload' or file_ext == '.part':
                    console.print("[red]File is still downloading (partial file detected)[/red]")
                    return {"status": "error", "reason": "Partial download detected"}
                
                console.print(f"[green]File size: {file_size} bytes, Extension: {file_ext}[/green]")
                
                s3 = boto3.client('s3', region_name=REGION)
                bucket_name = "your s3 bucket name" # replace with your S3 bucket name
                s3_file_key = f"downloaded-files/{client_name}/{os.path.basename(file_path)}"
                
                try:
                    s3.upload_file(file_path, bucket_name, s3_file_key)
                    presigned_url = s3.generate_presigned_url(
                        'get_object',
                        Params={'Bucket': bucket_name, 'Key': s3_file_key},
                        ExpiresIn=3600
                    )
                    console.print(f"✅ Uploaded to S3: {s3_file_key}")
                    return {
                        "output": {
                            "status": "success",
                            "s3_key": s3_file_key,
                            "s3_url": presigned_url,
                            "file_name": os.path.basename(file_path),
                            "file_size": file_size,
                            "method": "filesystem_check" if download_triggered else "event"
                        }
                    }
                except Exception as s3_error:
                    console.print(f"❌ Error uploading to S3: {repr(s3_error)}")
                    return {
                        "output": {
                            "status": "s3_error",
                            "reason": repr(s3_error)
                        }
                     }
            else:
                return {
                    "output": {
                        "status": "error",
                        "reason": "File not downloaded - all methods failed"
                    }
                }
                
    except Exception as e:
        console.print(f"[red]❌ Error in nova_act_download: {repr(e)}[/red]")
        return {
            "output": {
                "status": "error",
                "reason": f"NovaAct execution failed: {repr(e)}"
            }
        }


model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
model = BedrockModel(model_id=model_id)
agent = Agent(
    model=model,
    tools=[nova_act_download],
    system_prompt="""You are a helpful Web UI automation assistant.

IMPORTANT BEHAVIOR RULES:
- Do NOT retry failed tool calls on your own. If a tool fails, explain the error and stop.
- Never output the password in plain text in your responses.
- Never make multiple attempts to run the same web automation in a single response.

IMPORTANT:
After running a tool, you MUST produce a final assistant message containing ONLY the JSON returned by the tool.

You will receive structured data with these fields:
- Target Website URL, Login Username, Login Password, Task Instructions, Client Name

Follow this exact process:
1. Extract the website URL, login credentials, and task instructions
2. Use the nova_act_download tool with THREE arguments:
   a. instruction: "Login using username: {username} and password: {password}. Then {task instructions}."
   b. starting_url: The website URL
   c. client_name: The client identifier
3. IMPORTANT: Return ONLY the JSON response from the tool, do not add any additional text or summary.
   Return the exact dict that the tool returns with these fields: status, s3_key, s3_url, file_name, file_size, method
"""
)

@app.entrypoint
def invoke_agent(payload):
    """Process JSON payload and return structured result"""
    
    weburl = payload.get("weburl")
    username = payload.get("username")
    password = payload.get("password")
    promptfile = payload.get("promptfile")
    client_name = payload.get("client_name")
    
    if not all([weburl, username, password, promptfile, client_name]):
        return {"status": "error", "message": "Missing required fields"}
    
    prompt = f"""Execute web automation with these details:
- Website URL: {weburl}
- Username: {username}
- Password: {password}
- Task: {promptfile}
- Client: {client_name}
"""
    
    print("🚀 Invoking agent with Claude...")
    response = agent(prompt)
    # Debug: print the structure
    content = response.message["content"]
    print(f"✅ Agent response received")
    print(f"Response type: {type(content)}")
    print(f"Response content: {content}")
        
    # content is a LIST: [{'text': '{"status": "success", ...}'}]
    raw = content[0]["text"]
    print(f"Raw text: {raw}")
        
    # The text is JSON (double quotes), use json.loads
    try:
        result = json.loads(raw)
        print(f"✅ Parsed with json.loads: {result}")
    except json.JSONDecodeError:
        # Fallback to ast.literal_eval for Python dict format
        try:
            result = ast.literal_eval(raw)
            print(f"✅ Parsed with ast.literal_eval: {result}")
        except (ValueError, SyntaxError) as e:
            print(f"❌ Both parsers failed: {e}")
            return {"status": "error", "message": "Failed to parse response", "raw": raw[:200]}
        
    # Ensure we return a dict
    if isinstance(result, dict):
        return result
    else:
        return {"status": "error", "message": f"Unexpected type: {type(result).__name__}"}



print("✅ Agent setup complete!")
print(f"Model: {model_id}")
print(f"Region: {REGION}")
print("Ready to test with invoke_agent(payload)")
#if __name__ == "__main__":
#    app.run()

#### Invoking local agent

In [ ]:
# Add this cell after the agent setup (after cell #VSC-3ee04431)

import json

# Example payload for City of Austin Utilities
payload = {
    "weburl": "https://coautilities.com/wps/wcm/connect/occ/coa/home",
    "username": "your user name",
    "password": "your password",
    "promptfile": "After signing in, click view bill button and click on the view bill (pdf) button ONLY ONCE AND IMMEDIATELY RETURN ACTION COMPLETE",
    "client_name": "your client name"
}

# Invoke the agent directly
print("🚀 Starting web automation agent...")
result = invoke_agent(payload)

# Display the result
print("\n✅ Agent Execution Complete!\n")
print(json.dumps(result, indent=4))

## Preparing your agent for deployment on AgentCore Runtime

Let's now deploy our agents to AgentCore Runtime. To do so we need to:
* Import the Runtime App with `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
* Initialize the App in our code with `app = BedrockAgentCoreApp()`
* Decorate the invocation function with the `@app.entrypoint` decorator
* Let AgentCoreRuntime control the running of the agent with `app.run()`

### Strands Agents with Amazon Bedrock model
Let's start with our Strands Agent using Amazon Bedrock model. All the others will work exactly the same.

## What happens behind the scenes?

When you use `BedrockAgentCoreApp`, it automatically:

* Creates an HTTP server that listens on the port 8080
* Implements the required `/invocations` endpoint for processing the agent's requirements
* Implements the `/ping` endpoint for health checks (very important for asynchronous agents)
* Handles proper content types and response formats
* Manages error handling according to the AWS standards

## Deploying the agent to AgentCore Runtime

The `CreateAgentRuntime` operation supports comprehensive configuration options, letting you specify container images, environment variables and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent. 

**Note:** Operations best practice is to package code as container and push to ECR using CI/CD pipelines and IaC

In this tutorial can will the Amazon Bedrock AgentCore Python SDK to easily package your artifacts and deploy them to AgentCore runtime.

### Configure AgentCore Runtime deployment

First we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()
agent_name = "nabo_group_web_automation_agent"
response = agentcore_runtime.configure(
    entrypoint="first_stage_agent.py",
    auto_create_execution_role=False,
    execution_role="your agent's execution role arn", 
    ecr_repository="your ecr repository uri",#remember to use uri and remove tags
    auto_create_ecr=False,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response

### Launching agent to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [ ]:
launch_result = agentcore_runtime.launch(local_build=True)# local build will use docker to build the image locally

🔧 Local build mode: building locally, deploying to cloud (NEW OPTION!)
   • Build container locally with Docker
   • Deploy to Bedrock AgentCore cloud runtime
   • Requires Docker/Finch/Podman to be installed
   • Use when you need custom build control
Launching Bedrock AgentCore agent 'nabo_group_web_automation_agent' to cloud
Docker image built: bedrock_agentcore-nabo_group_web_automation_agent:latest
Using execution role from config: arn:aws:iam::254599367545:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-1891a66923
Uploading to ECR...
Using ECR repository from config: 254599367545.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-nabo_group_web_automation_agent
Authenticating with registry...
Registry authentication successful
Tagging image: bedrock_agentcore-nabo_group_web_automation_agent:latest -> 254599367545.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-nabo_group_web_automation_agent:latest
Pushing image to registry...


The push refers to repository [254599367545.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-nabo_group_web_automation_agent]
c910df5bdab5: Preparing
a718bf88574c: Preparing
967500daef21: Preparing
326564bf6b38: Preparing
4e86f5f42064: Preparing
af53caa366c8: Preparing
6dadb80ecd81: Preparing
ae0a4a38d5e0: Preparing
553c8f453276: Preparing
a1d3313d9587: Preparing
b79b8737a8f0: Preparing
6dadb80ecd81: Waiting
553c8f453276: Waiting
ae0a4a38d5e0: Waiting
a1d3313d9587: Waiting
af53caa366c8: Waiting
b79b8737a8f0: Waiting
4e86f5f42064: Layer already exists
a718bf88574c: Layer already exists
326564bf6b38: Layer already exists
967500daef21: Layer already exists
ae0a4a38d5e0: Layer already exists
af53caa366c8: Layer already exists
6dadb80ecd81: Layer already exists
553c8f453276: Layer already exists
b79b8737a8f0: Layer already exists
a1d3313d9587: Layer already exists
c910df5bdab5: Pushed


Image pushed successfully
Image uploaded to ECR: 254599367545.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-nabo_group_web_automation_agent
Deploying to Bedrock AgentCore...


latest: digest: sha256:fac535299762ed7b96f25896b8a34e6d2823d6773418a2852ebb72b71144a53b size: 2626


✅ Agent created/updated: arn:aws:bedrock-agentcore:us-east-1:254599367545:runtime/nabo_group_web_automation_agent-FQRoBmCqSM
Observability is enabled, configuring Transaction Search...
CloudWatch Logs resource policy already configured
X-Ray trace destination already configured
X-Ray indexing rule already configured
✅ Transaction Search already fully configured
🔍 GenAI Observability Dashboard:
   https://console.aws.amazon.com/cloudwatch/home?region=us-east-1#gen-ai-observability/agent-core
Polling for endpoint to be ready...
Agent endpoint: arn:aws:bedrock-agentcore:us-east-1:254599367545:runtime/nabo_group_web_automation_agent-FQRoBmCqSM/runtime-endpoint/DEFAULT
Deployed to cloud: arn:aws:bedrock-agentcore:us-east-1:254599367545:runtime/nabo_group_web_automation_agent-FQRoBmCqSM


### Checking for the AgentCore Runtime Status
Now that we've deployed the AgentCore Runtime, let's check for it's deployment status

In [78]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

Retrieved Bedrock AgentCore status for: nabo_group_web_automation_agent


'READY'

### Invoking AgentCore Runtime

Finally, we can invoke our AgentCore Runtime with a payload

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [ ]:
invoke_response = agentcore_runtime.invoke({
    "weburl": "your website url",
    "username": "your user name",
    "password": "your password",
    "promptfile": "After signing in, click view bill button and click on the view bill (pdf) button ONLY ONCE AND IMMEDIATELY RETURN ACTION COMPLETE",
    "client_name": "your client name"
})
invoke_response